In [ ]:
# Install required libraries
%pip install llama-index-llms-openai
!pip install llama-index

In [ ]:
# Import necessary libraries
import logging
import sys
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

# Configure logging
logging.basicConfig(stream=sys.stdout, level=logging.ERROR)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


In [ ]:
from llama_index.core import Settings


# Configure Ollama LLM
ollama_llm = Ollama(
    model="llama3.2:latest",
    base_url="http://localhost:11434",
    temperature=0.1
)

# Configure embedding model
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0}
)


Settings.llm = ollama_llm
Settings.embed_model = ollama_embedding


In [ ]:
# Load documents into the index
documents = SimpleDirectoryReader(input_files=['../data/2022 Q3 AAPL.pdf']).load_data()
index = VectorStoreIndex.from_documents(documents)

In [ ]:
# Create a retriever from the index
retriever = index.as_retriever(retriever_mode="default")  # Default mode for VectorStoreIndex

# Retrieve relevant nodes
query = "Provide the summary on the sales and profit"
nodes = retriever.retrieve(query)

# Display the retrieved nodes
print("Retrieved Nodes:")
for node in nodes:
    print(f"Node ID: {node.node_id}")
    print(f"Node Text: {node.text}\n")


In [ ]:
from llama_index.core import get_response_synthesizer

# Create a response synthesizer
synthesizer_simple = get_response_synthesizer(response_mode="simple_summarize", llm=ollama_llm)

# Create a query engine
query_engine_simple = RetrieverQueryEngine(retriever=retriever, response_synthesizer=synthesizer_simple)

# Execute query
response_simple = query_engine_simple.query(query)
print("Simple Summarize Response:")
print(response_simple)


In [ ]:
# Create a response synthesizer
synthesizer_refine = get_response_synthesizer(response_mode="refine", llm=ollama_llm)

# Create a query engine
query_engine_refine = RetrieverQueryEngine.from_args(retriever=retriever, response_synthesizer=synthesizer_refine)

# Execute query
response_refine = query_engine_refine.query(query)
print("Refine Response:")
print(response_refine)
